# Merge and Analyze CSV Data
This notebook merges two CSV files based on `nrElcom` and calculates the average remuneration (`power1 + eco1`) per canton and year.

In [75]:
# Import required libraries
import pandas as pd

In [76]:
# Load the CSV files
evu_df = pd.read_csv(r'c:\Users\André Winkler\OneDrive - FFHS\BINA\Git\LN_BINA_FS25\data\vese\evu_ergebnisse.csv', sep=';')
gemeinde_df = pd.read_csv(r'c:\Users\André Winkler\OneDrive - FFHS\BINA\Git\LN_BINA_FS25\data\vese\gemeinde_ergebnisse.csv', sep=';')

In [77]:
# Merge the dataframes on `nrElcom`
merged_df = pd.merge(evu_df, gemeinde_df, on='nrElcom', how='inner')

In [78]:
# Load Gemeindestand.csv to map cantons
gemeindestand_df = pd.read_csv(r'c:\Users\André Winkler\OneDrive - FFHS\BINA\Git\LN_BINA_FS25\data\vese\Gemeindestand.csv', sep=';')

# Map cantons using Gemeindestand.csv
gemeinde_to_kanton = gemeindestand_df.set_index('Gemeindename')['Kanton'].to_dict()
merged_df['Kanton'] = merged_df['Gemeinde'].map(gemeinde_to_kanton)

In [79]:
# Calculate the average remuneration per canton and year
# Fill missing values in 'energy1' and 'eco1' with 0
merged_df = merged_df.copy()  # Ensure we are working on a copy of the DataFrame
merged_df['energy1'] = merged_df['energy1'].fillna(0)
merged_df['eco1'] = merged_df['eco1'].fillna(0)

# Calculate remuneration and group by year and canton
merged_df['remuneration'] = merged_df['energy1'] + merged_df['eco1']
result = merged_df.groupby(['year', 'Kanton'])['remuneration'].mean().reset_index()
result.rename(columns={'remuneration': 'avg_remuneration'}, inplace=True)

In [80]:
# Display the result
print(result)

     year Kanton  avg_remuneration
0    2015     AG          0.000000
1    2015     AI          0.000000
2    2015     AR          0.000000
3    2015     BE          0.000000
4    2015     BL          0.000000
..    ...    ...               ...
255  2024     UR         13.457895
256  2024     VD         17.410000
257  2024     VS         14.531849
258  2024     ZG         21.004286
259  2024     ZH         18.762753

[260 rows x 3 columns]


In [82]:
# Import required libraries for visualization
import plotly.express as px

# Define a fixed order for cantons
fixed_canton_order = sorted(result['Kanton'].unique())

# Identify the top 4 cantons by average remuneration for each year
def assign_canton_group(row, top_cantons_per_year):
    if row['Kanton'] in top_cantons_per_year.get(row['year'], []):
        return row['Kanton']
    return 'Other'

# Get the top 4 cantons for each year
top_cantons_per_year = (
    result.groupby('year', group_keys=False)  # Exclude grouping columns
    .apply(lambda group: group.nlargest(4, 'avg_remuneration')['Kanton'].tolist(), include_groups=False)
    .to_dict()
)

# Assign canton group based on the top cantons per year
result['canton_group'] = result.apply(assign_canton_group, axis=1, top_cantons_per_year=top_cantons_per_year)

# Create an interactive bar chart with a year slider
fig = px.bar(result, x='Kanton', y='avg_remuneration', color='canton_group',
             animation_frame='year',
             title='Average Remuneration per Canton and Year',
             labels={'avg_remuneration': 'Average Remuneration (CHF)', 'Kanton': 'Canton'},
             category_orders={'Kanton': fixed_canton_order},  # Fix canton order
             color_discrete_map={
                 'Other': '#D9D9D9'  # Gray for other cantons
             },
             template='simple_white')

# Update layout for IBCS compliance, hide legend, and fix y-axis range
fig.update_layout(
    xaxis_title='Canton',
    yaxis_title='Average Remuneration (CHF)',
    title_font=dict(size=16, family='Arial', color='black'),
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=True, zeroline=True, zerolinecolor='black', range=[0, 31]),  # Fix y-axis range
    showlegend=False  # Hide legend
)

# Show the interactive plot
fig.show()